# Analysis

In [1]:
%load_ext autoreload
%autoreload 2

In [54]:
import matplotlib.pyplot as plt
import yt
from rich import print
from icecream import ic
import panel as pn
from loguru import logger

import logging
import json
from yt.data_objects.time_series import SimulationTimeSeries
from yt.data_objects.static_output import Dataset
from matplotlib.pyplot import Axes

In [27]:
dim = 1
# dim = 2
# dim = 3
beta = 0.25
theta = 60.0
plasma_resistivity = 100.0

In [28]:
import os
from pathlib import Path

try:
    base_dir = Path(os.getcwd()) / "01_oblique_linear_alfven"
    sub_dir = f"dim_{dim}_beta_{beta}_theta_{theta}_eta_{plasma_resistivity}"
    directory = base_dir / sub_dir
    os.chdir(directory)
    os.makedirs("figures", exist_ok=True)
except FileNotFoundError:
    pass

In [29]:
pn.extension()

# load simulation parameters
with open("sim_parameters.json", "rb") as f:
    meta = json.load(f)

ytLogger = logging.getLogger("yt")
ytLogger.setLevel(logging.WARNING)

In [30]:
ps = "ions"

if dim == 3:
    direction = "z"
elif dim == 2:
    # direction = "y"
    pass
else:
    direction = "x"

In [31]:
import numpy as np
import unyt as u
from utils import check_ds_type
from unyt.physical_constants import mu_0

def add_V_Alfven_field(ds: Dataset, direction: str):
    #: NOTE: rho is species charge density

    def _V_Alfven(field, data):
        rho_m = data["rho"] * meta["m_ion"] * u.m**(-3) / u.qp_mks.value * u.kg
        return data[f"B{direction}"] / np.sqrt(
            mu_0 * rho_m
        )

    name = ("boxlib", f"V_Alfven_{direction}")
    
    ds.add_field(name, function=_V_Alfven, units="km/s", sampling_type="local")


def add_V_field(ds: Dataset, direction: str):
    def _V(field, data):
        return data[f"particle_momentum_{direction}"] / meta["m_ion"] / u.kg

    name = (ps, f"V_{direction}")

    ds.add_field(name, function=_V, units="km/s", sampling_type="particle")


def add_field(ds: Dataset):
    type = check_ds_type(ds)
    if type == "particle":
        add_V_field(ds, "x")
        add_V_field(ds, "y")
        add_V_field(ds, "z")
    elif type == "field":
        add_V_Alfven_field(ds, "x")
        add_V_Alfven_field(ds, "y")
        add_V_Alfven_field(ds, "z")

## Fields

In [32]:
diag_format = meta['diag_format']

if diag_format == "openpmd":
    ext = meta['diag_openpmd_backend']
    file_format = f"/{diag_format}*.{ext}"
else:
    file_format = "??????"

In [33]:
field_diag_dir = "diags/diag1"
# field_diag_dir = "diags/chkpoint"
ts_field: SimulationTimeSeries = yt.load(f"{field_diag_dir}{file_format}")
ic(len(ts_field))
ds_field = ts_field[0]
# ts = yt.load('./diags/diag???0032')

ic| len(ts_field): 101


## Particles

In [34]:
part_diag_dir = "diags/diag2"

ts_part : SimulationTimeSeries = yt.load(f"{part_diag_dir}{file_format}")
ds_part : Dataset = ts_part[0]

In [35]:
def test_plot_part():

    x_field = f"particle_position_{direction}"
    y_field = "particle_momentum_y"

    p = yt.ParticlePlot(
        ds_part,
        x_field,
        y_field,
        (ps, "particle_weight"),
    )
    p.set_log(y_field, False)
    return p

# test_plot_part()

In [36]:
from functools import partial

if direction == "x":
    n_bins = ds_field.domain_dimensions[0]
elif direction == "y":
    n_bins = ds_field.domain_dimensions[1]
else:
    n_bins = ds_field.domain_dimensions[2]


weight_field=("boxlib", "volume")

plot_field = partial(
    yt.ProfilePlot,
    x_field=direction,
    weight_field=weight_field,
    x_log=False,
    y_log=False,
    n_bins=n_bins,
)

create_field_profile = partial(
    yt.create_profile,
    bin_fields=direction,
    weight_field=weight_field,
    n_bins=n_bins,
    deposition="cic",
)

In [37]:
# yt.create_profile()
# Note: since particle position is continuous, it makes sense to have more bins
particle_bins = 2 * n_bins

bin_fields = (ps, f"particle_position_{direction}")
weight_field = (ps, "particle_weight")

plot_particle_profile = partial(
    yt.ProfilePlot,
    x_field=bin_fields,
    weight_field=weight_field,
    x_log=False,
    y_log=False,
    n_bins=particle_bins,
)

field = "particle_momentum_y"

create_part_profile = partial(
    yt.create_profile,
    bin_fields=bin_fields,
    weight_field=weight_field,
    n_bins=particle_bins,
    deposition="cic",
)

In [38]:


# profile = create_part_profile(
    # ds_part.all_data(),
    # fields=[weight_field, field],
# )

In [56]:
def plot_field_with_plasma_profile(ds_field, ds_part, field0, field1, twin=True):
    
    field_profile = create_field_profile(ds_field.all_data(), fields = field0)
    part_profile = create_part_profile(ds_part.all_data(), fields=field1)
    
    p0 = yt.ProfilePlot.from_profiles(field_profile)
    p1 = yt.ProfilePlot.from_profiles(part_profile)
    p0.set_log(field0, False)
    p1.set_log(field1, False)

    # `ProfilePlot` does not support `deposition`
    # p0 = plot_field(ds_field, y_fields=field0)
    # p1 = plot_particle_profile(ds_part, y_fields=field1)

    # Customizing the plot
    fig, ax = plt.subplots()
    ax: Axes
    
    if twin:
        ax2 = ax.twinx()
    else:
        ax2 = ax

    plot = p0.plots[field0]
    plot.figure = fig
    plot.axes = ax

    plot = p1.plots[field1]
    plot.figure = fig
    plot.axes = ax2

    p0.render()
    p1.render()

    if twin:
        ax2.yaxis.set_label_position("right")
        ax2.yaxis.set_offset_position("right")
        # set the color of the labels and lines
        ax2.yaxis.label.set_color("orange")
        ax2.lines[0].set_color("orange")
        # ax2.set_ylim(ax.get_ylim())
        fig.set_size_inches(12, 5)
    
    ax.set_ylim(-200, 200)    
    ax2.set_ylim(-200, 200)
    return fig

def plot_field_with_plasma_profile_ts(
    ts_field,
    ts_part,
    name=None,
    step=8,
    field0=("boxlib", "V_Alfven_y"),
    field1=("V_y"),
):
    os.makedirs("figures/field_plasma_profile", exist_ok=True)
    for ds_field, ds_part in zip(ts_field[::step], ts_part[::step]):
        add_field(ds_field)
        add_field(ds_part)
        fig = plot_field_with_plasma_profile(ds_field, ds_part, field0, field1)
        fig.savefig(f"figures/field_plasma_profile/{name}_{ds_field.current_time}.png")
        plt.close(fig)


plot_field_with_plasma_profile_ts(ts_field, ts_part, name="v_comp", step=4)

In [40]:
#: check the field list
# print(ds_part.field_list)
# print(ds_part.derived_field_list)

In [ ]:
def test_add_field():
    fname = ds_part.add_deposited_particle_field(
        (ps, "particle_momentum_y"),
        method="nearest"
    )
    print(fname)
    print(ds_part.fluid_types)
    print(ds_part.particle_types)
    return fname


# YTFieldTypeNotFound: Could not find field type 'deposit'.

### Pressure

In [ ]:
import polars as pl
import polars.selectors as cs
import numpy as np

In [ ]:
# df["x"] = pd.qcut(df["particle_position_x"], q=10)

In [46]:
n = 64
if dim == 1 or dim == 2:
    i = 0
else:
    i = 2

breaks = np.linspace(ds_part.domain_left_edge[i], ds_part.domain_right_edge[i], ds_part.domain_dimensions[i])

def calc_pressure(ds, direction):
    
    df =  pl.DataFrame(ds2df(ds))
    
    return df.with_columns(
        # pl.col(f"particle_position_{direction}").qcut(n).alias(direction),
        pl.col(f"particle_position_{direction}").cut(breaks, include_breaks=True).alias(direction),
    ).with_columns(
        (cs.contains("momentum") - cs.contains("momentum").mean().over(direction))
        .pow(2)
        .name.suffix("_var"),
        # cs.contains("momentum").mean().over(direction).name.suffix("_mean"),
    ).group_by(direction).agg(
        cs.contains("var").sum(),
        pl.col("time").first()
    ).unnest(direction).rename({"brk": direction}).drop(cs.by_dtype(pl.Categorical))

In [ ]:
df_part = pl.concat(calc_pressure(ds, "z") for ds in ts_part)
df_part.write_ipc("part_pressure.arrow")

In [ ]:
df_part

In [ ]:
df_part.plot(x="z", y="time")

### yt

In [ ]:
ts: SimulationTimeSeries = yt.load('diags/diag1??????')
# ts = yt.load('./diags/diag???0032')

In [ ]:
def plot(ds, normalize = True, **kwargs):
    ad = ds.all_data()
    fields = ["Bx", "By", "Bz"]

    match meta["dim"]:
        case 1: pos = "x"
        case 2: pos = "y"
        
    pos = ad[pos]
    
    if normalize:
        pos = pos / meta['d_i']
    
    for field in fields:
        plt.plot(pos, ad[field], label=field, **kwargs)
    
    plt.xlabel("x ($d_i$)")
        
def hodogram(ds, comp1="By", comp2="Bz"):
    time = ds.current_time
    time_norm = time.value / meta['t_ci']
    ad = ds.all_data()
    plt.plot(ad[comp1], ad[comp2], label=f"t={time_norm:.2f}")
    plt.xlabel(comp1)
    plt.ylabel(comp2)

In [ ]:
for i, ds in enumerate(ts):
    alpha = (i + 1) / (len(ts)+1)
    plot(ds, alpha=alpha)
    plt.show()  # Show each plot separately

In [ ]:
i = 4
_ts = ts[0:i]
for i, ds in enumerate(_ts):
    alpha = (i + 1) / (len(_ts)+1)
    plot(ds, alpha=alpha)
    plt.show()  # Show each plot separately

In [ ]:
for ds in ts:
    hodogram(ds)
    plt.legend()

In [ ]:
i = 5
for ds in ts[0:i]:
    hodogram(ds)
    plt.legend()

In [ ]:
yt.SlicePlot(ds, "z", ("boxlib", "Bz"))

In [ ]:
ds.all_data()

In [ ]:
fields = [
    ("Bx"),
    ("By"),
    ("Bz"),
    ("mesh", "magnetic_field_strength"),
]

In [ ]:
for ds in ts.piter():
    p = yt.plot_2d(ds, fields=fields)
    p.set_log(fields, False)
    fig = p.export_to_mpl_figure((2, 2))
    fig.tight_layout()
    fig.savefig(f"figures/{ds}_magnetic_field.png")

### Average magnetic field

In [ ]:
def plot_avg(ds):
    fields = [
        ("Bx"),
        ("By"),
        ("Bz"),
    ]

    ad = ds.all_data()
    df = ad.to_dataframe(fields + ["x", zaxis])
    # compute the magnetic field strength
    df = df.assign(B=lambda x: (x.Bx**2 + x.By**2 + x.Bz**2) ** 0.5)

    axes = df.groupby(zaxis).mean().plot(y=fields + ["B"], subplots=True)
    return axes[0].figure

In [ ]:
def plot_avg_ts(i):
    return plot_avg(ts[i])
    
time_widget = pn.widgets.IntSlider(name="Time", value=1, start=0, end=len(ts)-1)
bound_plot = pn.bind(plot_avg_ts, i=time_widget)

pn.Column(time_widget, bound_plot)

In [ ]:
for ds in ts.piter():
    plot_avg(ds)

In [ ]:
ds.print_stats()
print(ds.field_list)

In [ ]:
grid = ds.r[:,:,:]
obj = grid.to_xarray(fields=fields)